# **การจำแนกที่ดินของปัญญาประดิษฐ์ (AI) **

ตัวอย่างจาก https://livingatlas.arcgis.com/landcover/

ที่มีอยู่ได้รับการปรับปรุงโดยการรวบรวมชุดข้อมูลการฝึกอบรมจำนวนมหาศาลซึ่งประกอบด้วยพิกเซลรูปภาพที่กำกับโดยมนุษย์หลายพันล้านภาพ แบบจำลองเหล่านี้ถูกนำไปใช้กับคอลเลกชันฉาก Sentinel-2 ทั้งหมดในแต่ละปีตั้งแต่ปี 2017 ถึง 2022 ซึ่งเป็นการสังเกตการณ์โลกมากกว่า 2,000,000 ครั้งจาก 6 แถบสเปกตรัมเพื่อสร้างแผนที่

ผลลัพธ์จะแสดงแผนที่พื้นผิว 9 ระดับได้แก่ vegetation, bare surface, water, cropland และ built area แผนที่เหล่านี้หาได้จาก ArcGIS Living Atlas of the World


In [1]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-bunyavee')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
#กำหนดขอบเขตพื้นที่ศึกษา
Map = geemap.Map()
Map.add_basemap('HYBRID')
thailand = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Thailand'))


# **ดึงข้อมูล LU/LC ของ ESRI**

จาก https://gee-community-catalog.org/ ซึ่งเป็น
Collection ของ "Samapriya Roy, Alec L. Robitaille, & Swetnam, T. (2023). samapriya/awesome-gee-community-datasets: Community Catalog (2.2.0).Zenodo. https://doi.org/10.5281/zenodo.10240803.


In [3]:
esri = ee.ImageCollection(
    'projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS'
)

esri_2017 = esri.filterDate('2017-01-01', '2017-12-31').mosaic()
esri_2018 = esri.filterDate('2018-01-01', '2018-12-31').mosaic()
esri_2019 = esri.filterDate('2019-01-01', '2019-12-31').mosaic()
esri_2020 = esri.filterDate('2020-01-01', '2020-12-31').mosaic()
esri_2021 = esri.filterDate('2021-01-01', '2021-12-31').mosaic()
esri_2022 = esri.filterDate('2022-01-01', '2022-12-31').mosaic()
esri_2023 = esri.filterDate('2023-01-01', '2023-12-31').mosaic()
esri_2024 = esri.filterDate('2024-01-01', '2024-12-31').mosaic()

esri_vis = {'min': 1, 'max': 11, 'palette': 'esri_lulc'}

Map.addLayer(esri_2017, esri_vis, 'ESRI LULC 2017')
Map.addLayer(esri_2018, esri_vis, 'ESRI LULC 2018')
Map.addLayer(esri_2019, esri_vis, 'ESRI LULC 2019')
Map.addLayer(esri_2020, esri_vis, 'ESRI LULC 2020')
Map.addLayer(esri_2021, esri_vis, 'ESRI LULC 2021')
Map.addLayer(esri_2022, esri_vis, 'ESRI LULC 2022')
Map.addLayer(esri_2022, esri_vis, 'ESRI LULC 2023')
Map.addLayer(esri_2022, esri_vis, 'ESRI LULC 2024')

Map.add_legend(title='ESRI Land Cover', builtin_legend='ESRI_LandCover_TS')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
images = ee.List([esri_2017, esri_2018, esri_2019, esri_2020, esri_2021, esri_2022, esri_2023, esri_2024])
collection = ee.ImageCollection.fromImages(images)
#list ข้อมูล
images

In [5]:
years = [str(year) for year in range(2017, 2025)]
#list ปีของข้อมูล
years

['2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

In [6]:
Map = geemap.Map()
Map.ts_inspector(collection, years, esri_vis, width='80px')
Map.add_legend(title='ESRI Land Cover', builtin_legend='ESRI_LandCover_TS')
Map.centerObject(thailand)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=Layo…

In [7]:
def mask_trees(img):
    return img.eq(2).clipToCollection(thailand).selfMask()

Thailand_trees = collection.map(mask_trees)
Map = geemap.Map()

trees_style = {
  'min': 0,
  'max': 1,
  'palette': ['#ab6c28']
}

Map.addLayer(Thailand_trees.first(), trees_style, 'trees Image')
Map.centerObject(thailand, 8)
Map


Map(center=[15.09165810341652, 101.012829945229], controls=(WidgetControl(options=['position', 'transparent_bg…

In [8]:
Map = geemap.Map()
Map.add_basemap('Hybrid')

trees_2017 = esri_2017.eq(2).clipToCollection(thailand).selfMask()
trees_2021 = esri_2022.eq(2).clipToCollection(thailand).selfMask()

trees_gain = esri_2017.neq(2).And(esri_2022.eq(2)).clipToCollection(thailand).selfMask()
trees_loss = esri_2017.eq(2).And(esri_2022.neq(2)).clipToCollection(thailand).selfMask()

Map.addLayer(trees_2017, {'palette': 'brown'}, 'trees 2017',False)
Map.addLayer(trees_2021, {'palette': 'cyan'}, 'trees 2022', False)

Map.addLayer(trees_gain, {'palette': 'yellow'}, 'trees gain')
Map.addLayer(trees_loss, {'palette': 'red'}, 'trees loss')
Map.centerObject(thailand,8)
Map

Map(center=[15.091658103416536, 101.012829945229], controls=(WidgetControl(options=['position', 'transparent_b…

# 1) จาก Code ข้างต้นให้นักศึกษา เขียนอธิบาย แนวคิด วิธีการ และขั้นตอนการทำงานของ Code ในแต่ละ Cell

# 2) **คำถาม**

จากข้อมูลข้างต้นให้นักศึกษา คิด วิเคราะห์ และตอบคำถามดังต่อไปนี้

1. เหตุใดจึงมีแนวโน้มที่จะมี Trees gain มากกว่า Trees Lose

1.) นโยบายการเพิ่มพื้นที่ป่าและการฟื้นฟูป่า (Reforestation & Afforestation)
ประเทศไทยมีนโยบาย เพิ่มพื้นที่ป่าไม้ให้ได้ 40% ของพื้นที่ประเทศ (กรมป่าไม้, 2565) รวมถึงมีโครงการปลูกป่าเฉลิมพระเกียรติ, ปลูกป่าในพื้นที่เสื่อมโทรม, และการทำ วนเกษตร
ทำให้ข้อมูลจากภาพถ่ายดาวเทียมอาจสะท้อน Trees gain มากขึ้น

2.) การฟื้นฟูป่าธรรมชาติ (Natural regeneration)
หลังจากพื้นที่ถูกทิ้งร้างทางเกษตรหรือหยุดทำไร่เลื่อนลอย พื้นที่เหล่านั้นมักเกิด การฟื้นตัวของพืชไม้ยืนต้น
งานวิจัยของ Chazdon (2014) พบว่าป่าเสื่อมโทรมสามารถฟื้นฟูกลับคืนได้อย่างรวดเร็วในเขตร้อน

3.) ข้อจำกัดด้านการตรวจจับการสูญเสีย (Tree Loss detection)
การใช้ภาพดาวเทียม เช่น Hansen Global Forest Change dataset อาจนับ Tree gain ได้ง่าย (เช่น การเห็นพื้นที่ที่เขียวขึ้น)
แต่การตรวจจับ Tree loss อาจคลาดเคลื่อน หากเกิดการตัดไม้แบบเลือกตัด (Selective logging) หรือไฟป่าในพื้นที่เล็กๆ ที่ดาวเทียมความละเอียดไม่พอ

2. นักศึกษาคิดว่า ข้อมูลดังกล่าวมีความน่าเชื่อถือหรือไม่อย่างไร ให้หาเอกสารอ้าวอิงเพื่อสนับสนุนแนวคิดของนักศึกษา และสรุปเพื่อตอบคำถาม

ข้อมูลป่าไม้จาก Remote Sensing เช่น Global Forest Watch (GFW), Hansen et al. (2013) เป็นที่ยอมรับในระดับนานาชาติ และใช้ในงานวิจัยจำนวนมาก

แต่มีข้อจำกัด คือ

- Resolution (30m pixel) อาจไม่ละเอียดพอในพื้นที่เล็กๆ

- Tree gain อาจรวมพื้นที่ปลูกเชิงพาณิชย์ เช่น ปลูกยูคาลิปตัส ยางพารา ปาล์มน้ำมัน ซึ่งไม่ใช่ป่าธรรมชาติ

- Tree loss อาจต่ำกว่าความจริง ถ้ามีการตัดแบบกระจายหรือตัดในพื้นที่ซ่อนเร้น

ดังนั้น ความน่าเชื่อถือถือว่า ค่อนข้างสูง แต่ต้องตีความด้วยความระมัดระวัง และควรใช้ ข้อมูลเสริม เช่น รายงานจากกรมป่าไม้ หรือข้อมูลภาคสนาม

เอกสารอ้างอิง

Hansen, M. C., et al. (2013). High-Resolution Global Maps of 21st-Century Forest Cover Change. Science, 342(6160), 850–853.

Chazdon, R. L. (2014). Second Growth: The Promise of Tropical Forest Regeneration in an Age of Deforestation. University of Chicago Press.

กรมป่าไม้ (2565). รายงานสถานการณ์ป่าไม้ประเทศไทย ปี 2565.